In [22]:
include("postgresql.jl")
include("TunnelingModule.jl")

##get training data
#get sequential data: 4 combinations of handedness
seq_ll_train = get_data(true, islhp=true, islhb=true, sorted=true)
seq_lr_train = get_data(true, islhp=true, islhb=false, sorted=true)
seq_rl_train = get_data(true, islhp=false, islhb=true, sorted=true)
seq_rr_train = get_data(true, islhp=false, islhb=false, sorted=true)
#get unsorted data: 8 combinations of handedness and ball type
#fast_ll_train = get_data(true, islhp=true, islhb=true)
#fast_lr_train = get_data(true, islhp=true, islhb=false)
#fast_rl_train = get_data(true, islhp=false, islhb=true)
#fast_rr_train = get_data(true, islhp=false, islhb=false)
#slow_ll_train = get_data(true, islhp=true, islhb=true, fast=false)
#slow_lr_train = get_data(true, islhp=true, islhb=false, fast=false)
#slow_rl_train = get_data(true, islhp=false, islhb=true, fast=false)
#slow_rr_train = get_data(true, islhp=false, islhb=false, fast=false)

##get test data
#get sequential data: 4 combinations of handedness
seq_ll_test = get_data(false, islhp=true, islhb=true, sorted=true)
seq_lr_test = get_data(false, islhp=true, islhb=false, sorted=true)
seq_rl_test = get_data(false, islhp=false, islhb=true, sorted=true)
seq_rr_test = get_data(false, islhp=false, islhb=false, sorted=true)
#get unsorted data: 8 combinations of handedness and ball type
#fast_ll_test = get_data(false, islhp=true, islhb=true)
#fast_lr_test = get_data(false, islhp=true, islhb=false)
#fast_rl_test = get_data(false, islhp=false, islhb=true)
#fast_rr_test = get_data(false, islhp=false, islhb=false)
#slow_ll_test = get_data(false, islhp=true, islhb=true, fast=false)
#slow_lr_test = get_data(false, islhp=true, islhb=false, fast=false)
#slow_rl_test = get_data(false, islhp=false, islhb=true, fast=false)
#slow_rr_test = get_data(false, islhp=false, islhb=false, fast=false)

##calculate distances and ratios
#sequential data
@show size(seq_ll_train[1])
seq_ll_ratios = calc_ratios(seq_ll_train, seq=true)
seq_lr_ratios = calc_ratios(seq_lr_train, seq=true)
seq_rl_ratios = calc_ratios(seq_rl_train, seq=true)
seq_rr_ratios = calc_ratios(seq_rr_train, seq=true)
#unsorted data
#fast_ll_ratios = calc_ratios(fast_ll_train)
#fast_lr_ratios = calc_ratios(fast_lr_train)
#fast_rl_ratios = calc_ratios(fast_rl_train)
#fast_rr_ratios = calc_ratios(fast_rr_train)
#slow_ll_ratios = calc_ratios(slow_ll_train)
#slow_lr_ratios = calc_ratios(slow_lr_train)
#slow_rl_ratios = calc_ratios(slow_rl_train)
#slow_rr_ratios = calc_ratios(slow_rr_train)

##prep test data

Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
size(seq_ll_train[1]) = (68012, 11)
size(breakdist) = (37955,)
size(Y) = (37955,)
size(b

([0.128808 0.292831 0.50612; 0.137515 0.0834631 1.23471; … ; 0.147052 0.63952 1.13918; 0.0921512 0.509186 0.934551], [1.72837 0.439873; 14.7935 1.64762; … ; 1.78131 0.229941; 1.83538 0.180977], [0.074526, 0.00929563, 0.0239039, 0.0584286, 0.0361307, 0.049257, 0.0174784, 0.0941057, 0.105359, 0.168225  …  0.178828, 0.105382, 0.165181, 0.012848, 0.100231, 0.186329, 0.0501107, 0.0220044, 0.0825527, 0.0502082], Any[-0.0655811, -0.145117, -0.0599418, 1.39201, -0.210698, 0.067862, -0.223813, 0.0384096, -0.0629139, 0.511992  …  -0.221142, -0.0599418, 0.0384096, 0.0471762, -0.376912, -0.0724712, -0.0599418, 0.0471762, -0.0724712, -0.223813])

In [36]:
##sequential neural net
#try ratios first
p = size(seq_ll_ratios[2],2)
    
loss_fn = mse
n = 16
m = Chain(Dense(p, n),Dense(n,n,sigmoid),Dense(n,1))  
loss(x, y) = loss_fn(m(x), y) 
evalcb = () -> @show([loss(seq_ll_ratios[2]',seq_ll_ratios[4]')])
opt = ADAM(params(m))
@time for idx = 1 : 500
    dataset = Base.Iterators.repeated((seq_ll_ratios[2]',seq_ll_ratios[4]'), 1)
    Flux.train!(loss, dataset, opt)
    if rem(idx,50) == 0
        @show loss(seq_ll_ratios[2]',seq_ll_ratios[4]')
    end
end

 
#Flux.train!(loss, dataset, opt, cb = throttle(evalcb, 0.01))

loss((seq_ll_ratios[2])', (seq_ll_ratios[4])') = 0.0499739132151547 (tracked)
loss((seq_ll_ratios[2])', (seq_ll_ratios[4])') = 0.047710822962806175 (tracked)
loss((seq_ll_ratios[2])', (seq_ll_ratios[4])') = 0.04691983898939782 (tracked)
loss((seq_ll_ratios[2])', (seq_ll_ratios[4])') = 0.04663800233161962 (tracked)
loss((seq_ll_ratios[2])', (seq_ll_ratios[4])') = 0.046512639092283904 (tracked)
loss((seq_ll_ratios[2])', (seq_ll_ratios[4])') = 0.04643470659352582 (tracked)
loss((seq_ll_ratios[2])', (seq_ll_ratios[4])') = 0.046372703093368044 (tracked)
loss((seq_ll_ratios[2])', (seq_ll_ratios[4])') = 0.04632095182282892 (tracked)
loss((seq_ll_ratios[2])', (seq_ll_ratios[4])') = 0.046278049263617345 (tracked)
loss((seq_ll_ratios[2])', (seq_ll_ratios[4])') = 0.04624341024299171 (tracked)
262.130324 seconds (1.69 G allocations: 64.567 GiB, 38.25% gc time)


In [38]:
p = size(seq_ll_ratios[2],2)
    
loss_fn = mse
n = 16
m = Chain(Dense(p, n),Dense(n,n,sigmoid),Dense(n,1))  
loss(x, y) = loss_fn(m(x), y) 
evalcb = () -> @show([loss(seq_ll_ratios[2]',seq_ll_ratios[4]')])
opt = ADAM(params(m))
dataset = Base.Iterators.repeated((seq_ll_ratios[2]',seq_ll_ratios[4]'), 500)
@time Flux.train!(loss, dataset, opt, cb = throttle(evalcb, 0.1))

[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[1.50026 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[1.33068 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[1.17448 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[1.03177 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.902498 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.786348 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.682853 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.591368 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.511147 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tra

[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.071692 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.0712784 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.0708679 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.0704603 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.0700556 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.0696536 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.0692543 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.0688577 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.0684639 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')

[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.0525207 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.0524171 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.0523157 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.0522165 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.0521193 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.0520241 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.0519309 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.0518396 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])')] = Flux.Tracker.TrackedReal{Float64}[0.0517502 (tracked)]
[loss((seq_ll_ratios[2])', (seq_ll_ratios[4])'